# Lab 6: Spark Streaming với Kafka Integration

## 🎯 **Learning Objectives:**
- Integrate Spark Structured Streaming với Kafka
- Create Kafka producers để send data
- Read streaming data từ Kafka topics
- Process real-time data với windows và watermarking
- Write results back to Kafka hoặc other sinks

## 📚 **Key Concepts:**
1. **Kafka Producer**: Send data to Kafka topics
2. **Spark Kafka Source**: Read from Kafka in Spark
3. **Schema Parsing**: Parse JSON/AVRO messages
4. **Real-time Processing**: Process streaming data with windows
5. **Kafka Sink**: Write results back to Kafka

## 🏗️ **Architecture Overview:**
```
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│  Kafka Producer │───▶│   Kafka Topic    │───▶│  Spark Streaming│
│  (Python/Java)  │    │   (user-events)  │    │  (readStream)   │
└─────────────────┘    └──────────────────┘    └────────┬────────┘
                                                          │
                                                          ▼
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│  Kafka Sink     │◀───│  Processed Data  │◀───│  Window/Aggregate│
│  (results)      │    │  (with watermark)│    │  (sliding/tumbling)│
└─────────────────┘    └──────────────────┘    └─────────────────┘
```

## 📊 **Use Cases:**
- **Real-time Analytics**: Process events from Kafka in real-time
- **Event Processing**: Transform and aggregate streaming events
- **Data Pipeline**: Build end-to-end streaming pipelines
- **Monitoring**: Real-time monitoring và alerting


In [ ]:
# Install and Import Dependencies
%pip install pyspark findspark pandas numpy pyarrow kafka-python

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.streaming import StreamingQuery
from kafka import KafkaProducer, KafkaConsumer
from kafka.admin import KafkaAdminClient, NewTopic
import json
import time
import threading
from datetime import datetime, timedelta
import random

print("✅ Dependencies installed and imported successfully!")


In [ ]:
# Initialize Spark Session for Kafka Streaming
spark = SparkSession.builder \
    .appName("SparkKafkaIntegration") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.streaming.checkpointLocation", "/tmp/kafka_checkpoint") \
    .config("spark.sql.streaming.kafka.useDeprecatedOffsetFetching", "false") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

# Set log level to reduce verbosity
spark.sparkContext.setLogLevel("WARN")

print("🚀 Spark Kafka Streaming Session initialized!")
print(f"📊 Spark Version: {spark.version}")
print(f"🔗 Master URL: {spark.sparkContext.master}")
print(f"📁 Checkpoint Location: /tmp/kafka_checkpoint")

# Kafka Configuration
KAFKA_BOOTSTRAP_SERVERS = "localhost:9092"
INPUT_TOPIC = "spark-lab-events"
OUTPUT_TOPIC = "spark-lab-results"

print(f"\n📡 Kafka Configuration:")
print(f"   Bootstrap Servers: {KAFKA_BOOTSTRAP_SERVERS}")
print(f"   Input Topic: {INPUT_TOPIC}")
print(f"   Output Topic: {OUTPUT_TOPIC}")


## Exercise 1: Setup Kafka Topics

### 🎯 **Learning Objectives:**
- Create Kafka topics for streaming
- Verify topic creation
- Understand topic configuration


In [ ]:
# Setup Kafka Topics
print("📋 Exercise 1: Setup Kafka Topics")
print("=" * 60)

try:
    # Create Kafka admin client
    admin_client = KafkaAdminClient(
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
        client_id='spark-lab-admin'
    )
    
    # Check if topics exist
    existing_topics = admin_client.list_topics()
    print(f"\n📊 Existing topics: {existing_topics}")
    
    # Create topics if they don't exist
    topics_to_create = []
    
    if INPUT_TOPIC not in existing_topics:
        topics_to_create.append(
            NewTopic(
                name=INPUT_TOPIC,
                num_partitions=3,
                replication_factor=1
            )
        )
        print(f"✅ Will create input topic: {INPUT_TOPIC}")
    else:
        print(f"ℹ️  Input topic already exists: {INPUT_TOPIC}")
    
    if OUTPUT_TOPIC not in existing_topics:
        topics_to_create.append(
            NewTopic(
                name=OUTPUT_TOPIC,
                num_partitions=3,
                replication_factor=1
            )
        )
        print(f"✅ Will create output topic: {OUTPUT_TOPIC}")
    else:
        print(f"ℹ️  Output topic already exists: {OUTPUT_TOPIC}")
    
    if topics_to_create:
        admin_client.create_topics(new_topics=topics_to_create, validate_only=False)
        print(f"\n✅ Created {len(topics_to_create)} topic(s)")
        time.sleep(2)  # Wait for topics to be ready
    else:
        print("\n✅ All topics already exist")
    
    # List all topics
    final_topics = admin_client.list_topics()
    print(f"\n📋 All available topics: {final_topics}")
    
except Exception as e:
    print(f"\n⚠️  Error setting up topics: {e}")
    print("   Note: Topics may need to be created manually or auto-creation is enabled")
    print("   You can create topics using:")
    print(f"   kafka-topics --create --bootstrap-server localhost:9092 --topic {INPUT_TOPIC} --partitions 3 --replication-factor 1")
    print(f"   kafka-topics --create --bootstrap-server localhost:9092 --topic {OUTPUT_TOPIC} --partitions 3 --replication-factor 1")


## Exercise 2: Create Kafka Producer

### 🎯 **Learning Objectives:**
- Create Kafka producer in Python
- Send JSON messages to Kafka topic
- Simulate real-time event stream
- Understand producer configuration


In [ ]:
# Create Kafka Producer
print("📤 Exercise 2: Create Kafka Producer")
print("=" * 60)

# Create producer
producer = KafkaProducer(
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    value_serializer=lambda v: json.dumps(v).encode('utf-8'),
    key_serializer=lambda k: k.encode('utf-8') if k else None
)

print("✅ Kafka Producer created!")

# Function to generate sample events
def generate_event(event_id, base_time=None):
    """Generate a sample event"""
    if base_time is None:
        base_time = datetime.now()
    
    event_time = base_time - timedelta(seconds=random.randint(0, 60))
    
    return {
        'event_id': f'EVT_{event_id:04d}',
        'user_id': f'user_{random.randint(1, 10)}',
        'event_type': random.choice(['click', 'view', 'purchase', 'add_to_cart']),
        'timestamp': event_time.strftime('%Y-%m-%d %H:%M:%S'),
        'amount': random.randint(10, 500) if random.random() > 0.6 else 0,
        'product_id': f'product_{random.randint(1, 20)}',
        'category': random.choice(['Electronics', 'Clothing', 'Books', 'Food'])
    }

# Send sample events
print("\n📤 Sending sample events to Kafka...")
base_time = datetime.now()

for i in range(20):
    event = generate_event(i, base_time)
    key = event['user_id']  # Use user_id as key for partitioning
    
    # Send to Kafka
    future = producer.send(INPUT_TOPIC, key=key, value=event)
    
    # Optional: wait for confirmation
    # record_metadata = future.get(timeout=10)
    # print(f"Sent: {event['event_id']} to partition {record_metadata.partition}")
    
    time.sleep(0.1)  # Small delay to simulate real-time stream

# Flush to ensure all messages are sent
producer.flush()
print(f"✅ Sent 20 events to topic: {INPUT_TOPIC}")

# Keep producer for later use
print("\n💡 Producer is ready. You can send more events later.")


In [ ]:
# Read from Kafka với Spark
print("📥 Exercise 3: Read from Kafka với Spark")
print("=" * 60)

# Define schema for events
event_schema = StructType([
    StructField("event_id", StringType()),
    StructField("user_id", StringType()),
    StructField("event_type", StringType()),
    StructField("timestamp", StringType()),
    StructField("amount", IntegerType()),
    StructField("product_id", StringType()),
    StructField("category", StringType())
])

# Read from Kafka
print(f"\n1️⃣ Reading from Kafka topic: {INPUT_TOPIC}")

kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS) \
    .option("subscribe", INPUT_TOPIC) \
    .option("startingOffsets", "earliest") \
    .option("failOnDataLoss", "false") \
    .load()

print("✅ Kafka stream created!")
print("\n📋 Kafka stream schema:")
kafka_stream.printSchema()

# Parse JSON from Kafka value
print("\n2️⃣ Parsing JSON messages:")

parsed_stream = kafka_stream \
    .select(
        col("key").cast("string").alias("kafka_key"),
        col("value").cast("string").alias("json_value"),
        col("timestamp").alias("kafka_timestamp"),
        col("partition"),
        col("offset")
    ) \
    .select(
        col("kafka_key"),
        from_json(col("json_value"), event_schema).alias("data"),
        col("kafka_timestamp"),
        col("partition"),
        col("offset")
    ) \
    .select(
        col("kafka_key"),
        col("data.*"),
        col("kafka_timestamp"),
        col("partition"),
        col("offset")
    ) \
    .withColumn(
        "event_timestamp",
        to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss")
    )

print("✅ Parsed stream schema:")
parsed_stream.printSchema()

# For demonstration, we'll use a memory sink to show data
print("\n3️⃣ Writing to memory sink to view data:")
print("   (In production, you'd use console, Kafka, or file sink)")

# Create a temporary view for querying
query = parsed_stream \
    .writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName("kafka_events") \
    .start()

# Wait a bit for data to arrive
print("\n⏳ Waiting for data to arrive...")
time.sleep(5)

# Query the memory table
if spark.catalog.tableExists("kafka_events"):
    print("\n📊 Sample events from Kafka:")
    spark.sql("SELECT * FROM kafka_events LIMIT 10").show(truncate=False)
else:
    print("\n⚠️  No data in memory table yet. Try sending more events.")

# Stop the query for now (we'll restart it in next exercises)
query.stop()
print("\n✅ Query stopped. Ready for next exercises.")


In [ ]:
# Windowed Aggregations với Kafka Stream
print("🪟 Exercise 4: Windowed Aggregations với Kafka Stream")
print("=" * 60)

# Recreate parsed stream with watermark
print("\n1️⃣ Creating stream with watermark:")

kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS) \
    .option("subscribe", INPUT_TOPIC) \
    .option("startingOffsets", "earliest") \
    .load()

parsed_with_watermark = kafka_stream \
    .select(
        from_json(col("value").cast("string"), event_schema).alias("data")
    ) \
    .select("data.*") \
    .withColumn(
        "event_timestamp",
        to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss")
    ) \
    .withWatermark("event_timestamp", "2 minutes")  # 2-minute watermark

print("✅ Stream created with 2-minute watermark")

# Windowed aggregation
print("\n2️⃣ Tumbling window aggregation (2-minute windows):")

windowed_agg = parsed_with_watermark \
    .groupBy(
        window(col("event_timestamp"), "2 minutes"),
        col("event_type")
    ) \
    .agg(
        count("*").alias("event_count"),
        countDistinct("user_id").alias("unique_users"),
        sum("amount").alias("total_revenue"),
        avg("amount").alias("avg_amount")
    ) \
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        col("event_type"),
        col("event_count"),
        col("unique_users"),
        col("total_revenue"),
        col("avg_amount")
    )

print("✅ Windowed aggregation defined")

# Write to console for demonstration
print("\n3️⃣ Starting streaming query (console output):")
print("   This will show results as windows complete")

query = windowed_agg \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate", "false") \
    .trigger(processingTime='5 seconds') \
    .start()

print("✅ Query started!")
print("\n💡 Send more events to Kafka to see windowed aggregations")
print("   Query will run until stopped")

# Note: In notebook, query runs in background
# You can check results in console or stop it
time.sleep(10)  # Let it run for a bit

print("\n⚠️  Query is running. To stop it, run: query.stop()")


In [ ]:
# Sliding Windows với Kafka
print("🔄 Exercise 5: Sliding Windows với Kafka")
print("=" * 60)

# Create stream with watermark
kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS) \
    .option("subscribe", INPUT_TOPIC) \
    .option("startingOffsets", "earliest") \
    .load()

parsed_stream = kafka_stream \
    .select(
        from_json(col("value").cast("string"), event_schema).alias("data")
    ) \
    .select("data.*") \
    .withColumn(
        "event_timestamp",
        to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss")
    ) \
    .withWatermark("event_timestamp", "5 minutes")

print("\n1️⃣ Sliding window: 5-minute window, 1-minute slide")

sliding_window_agg = parsed_stream \
    .groupBy(
        window(col("event_timestamp"), "5 minutes", "1 minute"),  # 5-min window, 1-min slide
        col("category")
    ) \
    .agg(
        count("*").alias("event_count"),
        countDistinct("user_id").alias("unique_users"),
        sum("amount").alias("total_revenue")
    ) \
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        col("category"),
        col("event_count"),
        col("unique_users"),
        col("total_revenue")
    )

print("✅ Sliding window aggregation defined")
print("\n📊 Window structure:")
print("   Window 1: [10:00 - 10:05)")
print("   Window 2: [10:01 - 10:06)  ← overlaps with Window 1")
print("   Window 3: [10:02 - 10:07)  ← overlaps with Window 1 & 2")

# Write to console
sliding_query = sliding_window_agg \
    .writeStream \
    .outputMode("update") \
    .format("console") \
    .option("truncate", "false") \
    .trigger(processingTime='5 seconds') \
    .start()

print("\n✅ Sliding window query started!")
print("💡 Send events to see overlapping windows in action")


## Exercise 6: Write Results to Kafka

### 🎯 **Learning Objectives:**
- Write processed results back to Kafka
- Configure Kafka sink
- Understand output format
- Build end-to-end Kafka pipeline


In [ ]:
# Write Results to Kafka
print("📤 Exercise 6: Write Results to Kafka")
print("=" * 60)

# Create stream and process
kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS) \
    .option("subscribe", INPUT_TOPIC) \
    .option("startingOffsets", "earliest") \
    .load()

parsed_stream = kafka_stream \
    .select(
        from_json(col("value").cast("string"), event_schema).alias("data")
    ) \
    .select("data.*") \
    .withColumn(
        "event_timestamp",
        to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss")
    ) \
    .withWatermark("event_timestamp", "2 minutes")

# Aggregate
aggregated = parsed_stream \
    .groupBy(
        window(col("event_timestamp"), "2 minutes"),
        col("event_type")
    ) \
    .agg(
        count("*").alias("event_count"),
        sum("amount").alias("total_revenue")
    ) \
    .select(
        col("window.start").cast("string").alias("window_start"),
        col("window.end").cast("string").alias("window_end"),
        col("event_type"),
        col("event_count"),
        col("total_revenue")
    )

print("\n1️⃣ Writing aggregated results to Kafka topic:")
print(f"   Output topic: {OUTPUT_TOPIC}")

# Convert to JSON string for Kafka
output_stream = aggregated \
    .select(
        to_json(struct([
            col("window_start"),
            col("window_end"),
            col("event_type"),
            col("event_count"),
            col("total_revenue")
        ])).alias("value")
    )

# Write to Kafka
kafka_sink_query = output_stream \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS) \
    .option("topic", OUTPUT_TOPIC) \
    .option("checkpointLocation", "/tmp/kafka_sink_checkpoint") \
    .outputMode("update") \
    .trigger(processingTime='5 seconds') \
    .start()

print("✅ Kafka sink query started!")
print(f"   Results are being written to: {OUTPUT_TOPIC}")
print("\n💡 You can consume from output topic to verify results")
print("   Example consumer code is shown below")


## Exercise 7: Consume Results from Kafka

### 🎯 **Learning Objectives:**
- Create Kafka consumer to read results
- Verify processed data
- Understand consumer groups
- Monitor streaming pipeline


In [ ]:
# Consume Results from Kafka
print("📥 Exercise 7: Consume Results from Kafka")
print("=" * 60)

# Create Kafka consumer
consumer = KafkaConsumer(
    OUTPUT_TOPIC,
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
    consumer_timeout_ms=5000,  # Timeout after 5 seconds
    auto_offset_reset='earliest',
    group_id='spark-lab-consumer'
)

print(f"✅ Consumer created for topic: {OUTPUT_TOPIC}")
print("\n📊 Consuming messages (timeout: 5 seconds):")

results = []
try:
    for message in consumer:
        result = message.value
        results.append(result)
        print(f"   Window: {result.get('window_start')} - {result.get('window_end')}")
        print(f"   Event Type: {result.get('event_type')}")
        print(f"   Count: {result.get('event_count')}, Revenue: {result.get('total_revenue')}")
        print()
        
        if len(results) >= 5:  # Limit to 5 messages for demo
            break
except Exception as e:
    print(f"   No more messages or error: {e}")

consumer.close()

if results:
    print(f"\n✅ Consumed {len(results)} result(s) from Kafka")
    print("\n📋 Summary of results:")
    for i, result in enumerate(results, 1):
        print(f"   {i}. {result.get('event_type')}: {result.get('event_count')} events, ${result.get('total_revenue')}")
else:
    print("\n⚠️  No results consumed. Make sure:")
    print("   1. Spark query is running and processing data")
    print("   2. Data has been sent to input topic")
    print("   3. Windows have completed (check watermark)")


## Exercise 8: Continuous Producer (Background Thread)

### 🎯 **Learning Objectives:**
- Create continuous event producer
- Run producer in background thread
- Simulate real-time event stream
- Test streaming pipeline end-to-end


In [ ]:
# Continuous Producer (Background Thread)
print("🔄 Exercise 8: Continuous Producer (Background Thread)")
print("=" * 60)

# Global flag to control producer
producer_running = False
producer_thread = None

def continuous_producer():
    """Continuously send events to Kafka"""
    global producer_running
    
    producer = KafkaProducer(
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
        value_serializer=lambda v: json.dumps(v).encode('utf-8'),
        key_serializer=lambda k: k.encode('utf-8') if k else None
    )
    
    event_id = 0
    base_time = datetime.now()
    
    while producer_running:
        event_id += 1
        event = generate_event(event_id, base_time)
        key = event['user_id']
        
        producer.send(INPUT_TOPIC, key=key, value=event)
        
        if event_id % 10 == 0:
            print(f"   📤 Sent {event_id} events...")
        
        time.sleep(1)  # Send 1 event per second
    
    producer.flush()
    producer.close()
    print("   ✅ Producer stopped")

def start_producer():
    """Start continuous producer in background"""
    global producer_running, producer_thread
    
    if producer_running:
        print("⚠️  Producer is already running")
        return
    
    producer_running = True
    producer_thread = threading.Thread(target=continuous_producer, daemon=True)
    producer_thread.start()
    print("✅ Continuous producer started in background")
    print("   Sending 1 event per second to Kafka")
    print("   Run stop_producer() to stop")

def stop_producer():
    """Stop continuous producer"""
    global producer_running
    
    if not producer_running:
        print("⚠️  Producer is not running")
        return
    
    producer_running = False
    if producer_thread:
        producer_thread.join(timeout=2)
    print("✅ Producer stopped")

print("\n💡 Functions available:")
print("   - start_producer(): Start sending events continuously")
print("   - stop_producer(): Stop the producer")
print("\n📝 Example usage:")
print("   start_producer()  # Start sending events")
print("   # ... run your Spark streaming queries ...")
print("   stop_producer()   # Stop when done")


## Summary

### ✅ What we learned:
1. **Kafka Topic Setup**: Create and configure Kafka topics
2. **Kafka Producer**: Send events to Kafka topics from Python
3. **Spark Kafka Source**: Read streaming data from Kafka
4. **Schema Parsing**: Parse JSON messages from Kafka
5. **Windowed Aggregations**: Apply windows và watermarking to Kafka streams
6. **Sliding Windows**: Use overlapping windows với real Kafka data
7. **Kafka Sink**: Write processed results back to Kafka
8. **Kafka Consumer**: Consume và verify results
9. **Continuous Producer**: Simulate real-time event streams

### 🎯 Key Takeaways:
- **Kafka Integration**: Spark can read và write to Kafka seamlessly
- **Real-time Processing**: Process events as they arrive from Kafka
- **Watermarking**: Essential for handling late data in Kafka streams
- **End-to-End Pipeline**: Complete pipeline from Kafka → Spark → Kafka
- **Producer/Consumer**: Use Python Kafka library for testing và monitoring

### 🚀 Best Practices:
- Use appropriate Kafka partitions for parallelism
- Set watermark based on expected Kafka message delays
- Monitor Kafka consumer lag
- Use checkpointing for fault tolerance
- Test với continuous producer for realistic scenarios
- Monitor Spark streaming query metrics

### 📚 Next Steps:
- Experiment with different window sizes và slides
- Test watermark behavior với intentionally late Kafka messages
- Monitor Kafka consumer groups và offsets
- Explore Kafka Schema Registry integration
- Build more complex pipelines với multiple Kafka topics
